In [33]:
# Load LSTM network and generate text
import sys
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.models import load_model
import h5py
import re
from keras.models import load_model
import tensorflow as tf
import pandas as pd
import math
from pprint import pprint

In [2]:
import platform
platform.architecture()[0]

'64bit'

In [3]:
mov_lines_file=open("/home/kavin/Silo/College Work/AI/DataSets/cornell_movie_dialogs_corpus/cornell movie-dialogs corpus/movie_lines.txt",'r')

In [4]:
mov_lines=mov_lines_file.readlines()

In [5]:
dialogues=[]
for mov_line in mov_lines:
    matched=re.match('.* \+\+\+\$\+\+\+ (.+)', mov_line)
    dialogues.append(matched.group(1))

In [ ]:
dialogues[1]

In [6]:
tokenized_dialogues=[]
for line in dialogues:
    tokenized_dialogues.append(filter(None,re.split(' |!|:|;|\?|"|\-\-|\*|<.*?>|</.*?>', line.lower())))

In [ ]:
tokenized_dialogues[0]

In [7]:
cleaned_dialogues=[]
for line in tokenized_dialogues:
    line_list=[]
    for token in line:
        try:
            matched=re.match('([\*|\'|_|\-|\[]|)(.*)([\*|\'|_|\-|\]]|)', token)
            line_list.append(matched.group(2))
        except:
            pass
    cleaned_dialogues.append(line_list)

In [ ]:
cleaned_dialogues[0]

In [8]:
all_dialogues=[]
for line in cleaned_dialogues:
    all_dialogues+=line
tokens=pd.Series(all_dialogues).unique()

In [56]:
bag_of_words={}
index_to_words={}
index=1
for word in tokens:
    bag_of_words[word]=index
    index_to_words[index]=word
    index+=1
#n_chars = len(tokenized)
n_vocab = len(tokens)

In [62]:
import csv

with open("gen_bag_of_words.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows([bag_of_words.keys(), bag_of_words.values()])
with open("gen_index_to_words.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows([index_to_words.keys(), index_to_words.values()])

In [26]:
bag_of_words

{'': 284,
 'fawn': 67833,
 'petra,': 110346,
 'considered.': 59177,
 'body-guard.': 36841,
 'considered,': 71798,
 'gah': 31325,
 'woods': 20038,
 'sending.': 62178,
 'spiders': 63796,
 'hanging': 5748,
 'woody': 63466,
 'hastily': 66633,
 "bringin'": 27587,
 "wainwright...'": 93122,
 'localized': 55707,
 'spidery': 46515,
 'tipsy.': 24289,
 'tipsy,': 34235,
 'know...right': 89884,
 'chatter': 44782,
 'course...': 23788,
 'cake,': 13623,
 'crossbar': 56804,
 'full-blooded,': 87885,
 'faggot,': 16401,
 'undead.': 10507,
 'undead,': 27007,
 'goodnight.': 18394,
 'more...logical': 88649,
 'can\t\t\t\t': 107192,
 'overcharge': 97045,
 'wax.': 95288,
 "wood'": 62892,
 'spider.': 53203,
 'donnie...but': 88940,
 'wood,': 27061,
 'wood.': 31534,
 'bogyman': 82768,
 'impotent.': 52015,
 'zulu.': 110622,
 "hangin'": 15399,
 'steak...': 95783,
 'bringing': 7396,
 'stronger...': 55182,
 "what'sit.": 103580,
 'n-nobody...': 107859,
 'elevates': 95233,
 'jackasses.': 108239,
 'of...shit.': 12264,
 '

In [ ]:
cleaned_dialogues[:10]

In [10]:
indexed_dialogues=[]
for line in cleaned_dialogues:
    line_list=[]
    for token in line:
        try:
            line_list.append(bag_of_words[token])
        except:
            line_list.append('0')
    indexed_dialogues.append(line_list)

In [ ]:
len(indexed_dialogues[1])

In [11]:
dialogues_dict={}
line_number=0
for line in indexed_dialogues:
    dialogues_dict[line_number]=line
    line_number+=1

In [12]:
all_keys=dialogues_dict.keys()

In [13]:
reply_keys=[]
response_keys=[]
for key in all_keys:
    if(key%2==0):
        reply_keys.append(key)
    else:
        response_keys.append(key)

In [ ]:
reply_index=0
response_index=1


In [ ]:
dialogues_df.head()

In [14]:
all_lines=[]
all_responses=[]

In [15]:
all_lines=list(np.array(indexed_dialogues)[reply_keys])
all_responses=list(np.array(indexed_dialogues)[response_keys])

In [ ]:
total_length=0
for line in all_lines:
    total_length+=len(line)
print total_length/len(all_lines)

In [ ]:
total_length=0
for line in all_responses:
    total_length+=len(line)
print total_length/len(all_responses)

In [60]:
len(all_lines)

152357

In [16]:
sample_lines=[]
sample_responses=[]
for line in all_lines[20000:]:
    sample_lines+=line
    if(len(sample_lines)>=10000):
        break
for line in all_responses[20000:]:
    sample_responses+=line
    if(len(sample_responses)>=10000):
        break

In [33]:
print(len(sample_lines))
print(len(sample_responses))

20003
20007


In [17]:
seq_length = 10
dataX = []
dataY = []
input_lines=[]
output_line=[]
for i in range(0, 10000-seq_length, 1):
    input_line=sample_lines[i:i+seq_length]
    output_line=sample_responses[i+seq_length]
    dataX.append(input_line)
    dataY.append(output_line)
n_patterns = len(dataX)
print "Total Patterns: ", n_patterns

Total Patterns:  9990


In [59]:
import json
with open('gen_bag_of_words.json', 'w', encoding = "ISO-8859-1") as fp:
    json.dump(bag_of_words, fp)
with open('gen_index_to_words.json', 'w',encoding = "ISO-8859-1") as fp:
    json.dump(index_to_words, fp)

TypeError: 'encoding' is an invalid keyword argument for this function

In [18]:
len(max(dataX, key=len))

84774

In [41]:
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (len(dataX), 10, 1))
    # normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [49]:
# define the LSTM model
model = Sequential()
model.add(LSTM(512, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [50]:
model.save("Donna Files/text_generator_model.h5")

In [46]:
# define the checkpoint
filepath="Donna Files/text_512_lstm_gen_weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
model.fit(X, y, epochs=1000, batch_size=128, callbacks=callbacks_list)

Epoch 1/1000
9990/9990 [==============================] - 29s - loss: 7.9189    
Epoch 2/1000
9990/9990 [==============================] - 29s - loss: 6.7449    
Epoch 3/1000
9990/9990 [==============================] - 28s - loss: 6.6249    
Epoch 4/1000
9990/9990 [==============================] - 29s - loss: 6.5773    
Epoch 5/1000
9990/9990 [==============================] - 29s - loss: 6.5550    
Epoch 6/1000
9990/9990 [==============================] - 29s - loss: 6.5440    
Epoch 7/1000
9990/9990 [==============================] - 29s - loss: 6.5428    
Epoch 8/1000
9990/9990 [==============================] - 29s - loss: 6.5421    
Epoch 9/1000
9990/9990 [==============================] - 29s - loss: 6.5369    
Epoch 10/1000
9990/9990 [==============================] - 29s - loss: 6.5340    
Epoch 11/1000
9990/9990 [==============================] - 29s - loss: 6.5316    
Epoch 12/1000
9990/9990 [==============================] - 28s - loss: 6.5318    
Epoch 13/1000
9990/9990 [

9990/9990 [==============================] - 28s - loss: 6.1526    
Epoch 65/1000
9990/9990 [==============================] - 28s - loss: 6.1139    
Epoch 66/1000
9990/9990 [==============================] - 28s - loss: 6.0736    
Epoch 67/1000
9990/9990 [==============================] - 29s - loss: 6.0233    
Epoch 68/1000
9990/9990 [==============================] - 28s - loss: 5.9835    
Epoch 69/1000
9990/9990 [==============================] - 28s - loss: 5.9378    
Epoch 70/1000
9990/9990 [==============================] - 28s - loss: 5.8920    
Epoch 71/1000
9990/9990 [==============================] - 28s - loss: 5.8348    
Epoch 72/1000
9990/9990 [==============================] - 29s - loss: 5.7873    
Epoch 73/1000
9990/9990 [==============================] - 29s - loss: 5.7265    
Epoch 74/1000
9990/9990 [==============================] - 28s - loss: 5.6711    
Epoch 75/1000
9990/9990 [==============================] - 28s - loss: 5.6087    
Epoch 76/1000
9990/9990 [=====

9990/9990 [==============================] - 28s - loss: 3.1703    
Epoch 122/1000
9990/9990 [==============================] - 28s - loss: 3.1572    
Epoch 123/1000
9990/9990 [==============================] - 28s - loss: 3.0759    
Epoch 124/1000
9990/9990 [==============================] - 28s - loss: 3.0727    
Epoch 125/1000
9990/9990 [==============================] - 28s - loss: 3.0057    
Epoch 126/1000
9990/9990 [==============================] - 28s - loss: 2.9894    
Epoch 127/1000
9990/9990 [==============================] - 28s - loss: 2.9456    
Epoch 128/1000
9990/9990 [==============================] - 28s - loss: 2.9143    
Epoch 129/1000
9990/9990 [==============================] - 28s - loss: 2.8994    
Epoch 130/1000
9990/9990 [==============================] - 28s - loss: 2.8628    
Epoch 131/1000
9990/9990 [==============================] - 28s - loss: 2.8434    
Epoch 132/1000
9990/9990 [==============================] - 28s - loss: 2.7798    
Epoch 133/1000
9990

9990/9990 [==============================] - 28s - loss: 1.7316    
Epoch 182/1000
9990/9990 [==============================] - 28s - loss: 1.7599    
Epoch 183/1000
9990/9990 [==============================] - 29s - loss: 1.7270    
Epoch 184/1000
9990/9990 [==============================] - 28s - loss: 1.6776    
Epoch 185/1000
9990/9990 [==============================] - 28s - loss: 1.6924    
Epoch 186/1000
9990/9990 [==============================] - 29s - loss: 1.6752    
Epoch 187/1000
9990/9990 [==============================] - 28s - loss: 1.6675    
Epoch 188/1000
9990/9990 [==============================] - 28s - loss: 1.6544    
Epoch 189/1000
9990/9990 [==============================] - 28s - loss: 1.6103    
Epoch 190/1000
9990/9990 [==============================] - 28s - loss: 1.5902    
Epoch 191/1000
9990/9990 [==============================] - 28s - loss: 1.6106    
Epoch 192/1000
9990/9990 [==============================] - 28s - loss: 1.5819    
Epoch 193/1000
9990

9990/9990 [==============================] - 28s - loss: 0.9943    
Epoch 249/1000
9990/9990 [==============================] - 28s - loss: 0.9780    
Epoch 250/1000
9990/9990 [==============================] - 28s - loss: 0.9724    
Epoch 251/1000
9990/9990 [==============================] - 28s - loss: 0.9622    
Epoch 252/1000
9990/9990 [==============================] - 28s - loss: 0.9802    
Epoch 253/1000
9990/9990 [==============================] - 28s - loss: 0.9442    
Epoch 254/1000
9990/9990 [==============================] - 28s - loss: 0.9585    
Epoch 255/1000
9990/9990 [==============================] - 28s - loss: 0.9865    
Epoch 256/1000
9990/9990 [==============================] - 28s - loss: 0.9402    
Epoch 257/1000
9990/9990 [==============================] - 28s - loss: 0.9305    
Epoch 258/1000
9990/9990 [==============================] - 28s - loss: 0.9351    
Epoch 259/1000
9990/9990 [==============================] - 28s - loss: 0.9237    
Epoch 260/1000
9990

...

9990/9990 [==============================] - 28s - loss: 0.1694    
Epoch 729/1000
9990/9990 [==============================] - 28s - loss: 0.0884    
Epoch 730/1000
9990/9990 [==============================] - 28s - loss: 0.0928    
Epoch 731/1000
9990/9990 [==============================] - 28s - loss: 0.0775    
Epoch 732/1000
9990/9990 [==============================] - 28s - loss: 0.0836    
Epoch 733/1000
9990/9990 [==============================] - 28s - loss: 0.0799    
Epoch 734/1000
9990/9990 [==============================] - 28s - loss: 0.0730    
Epoch 735/1000
9990/9990 [==============================] - 28s - loss: 0.0954    
Epoch 736/1000
9990/9990 [==============================] - 28s - loss: 0.0768    
Epoch 737/1000
9990/9990 [==============================] - 28s - loss: 0.0790    
Epoch 738/1000
9990/9990 [==============================] - 28s - loss: 0.0699    
Epoch 739/1000
9990/9990 [==============================] - 28s - loss: 0.0834    
Epoch 740/1000
9990

9990/9990 [==============================] - 28s - loss: 0.0667    
Epoch 772/1000
9990/9990 [==============================] - 28s - loss: 0.0704    
Epoch 773/1000
9990/9990 [==============================] - 28s - loss: 0.0850    
Epoch 774/1000
9990/9990 [==============================] - 28s - loss: 0.0773    
Epoch 775/1000
9990/9990 [==============================] - 28s - loss: 0.1833    
Epoch 776/1000
9990/9990 [==============================] - 28s - loss: 0.0817    
Epoch 777/1000
9990/9990 [==============================] - 28s - loss: 0.0774    
Epoch 778/1000
9990/9990 [==============================] - 28s - loss: 0.0650    
Epoch 779/1000
9990/9990 [==============================] - 28s - loss: 0.0690    
Epoch 780/1000
9990/9990 [==============================] - 28s - loss: 0.0681    
Epoch 781/1000
9990/9990 [==============================] - 28s - loss: 0.0801    
Epoch 782/1000
9990/9990 [==============================] - 28s - loss: 0.0786    
Epoch 783/1000
9990

9990/9990 [==============================] - 28s - loss: 0.2218    
Epoch 859/1000
9990/9990 [==============================] - 28s - loss: 0.0659    
Epoch 860/1000
9990/9990 [==============================] - 28s - loss: 0.0502    
Epoch 861/1000
9990/9990 [==============================] - 28s - loss: 0.0499    
Epoch 862/1000
9990/9990 [==============================] - 28s - loss: 0.0479    
Epoch 863/1000
9990/9990 [==============================] - 28s - loss: 0.0665    
Epoch 864/1000
9990/9990 [==============================] - 28s - loss: 0.0550    
Epoch 865/1000
9990/9990 [==============================] - 28s - loss: 0.0532    
Epoch 866/1000
9990/9990 [==============================] - 28s - loss: 0.0477    
Epoch 867/1000
9990/9990 [==============================] - 28s - loss: 0.0563    
Epoch 868/1000
9990/9990 [==============================] - 28s - loss: 0.1104    
Epoch 869/1000
9990/9990 [==============================] - 28s - loss: 0.0630    
Epoch 870/1000
9990

9990/9990 [==============================] - 28s - loss: 0.0401    
Epoch 947/1000
9990/9990 [==============================] - 28s - loss: 0.0394    
Epoch 948/1000
9990/9990 [==============================] - 28s - loss: 0.1254    
Epoch 949/1000
9990/9990 [==============================] - 28s - loss: 0.0974    
Epoch 950/1000
9990/9990 [==============================] - 28s - loss: 0.0552    
Epoch 951/1000
9990/9990 [==============================] - 28s - loss: 0.0518    
Epoch 952/1000
9990/9990 [==============================] - 28s - loss: 0.0900    
Epoch 953/1000
9990/9990 [==============================] - 28s - loss: 0.0500    
Epoch 954/1000
9990/9990 [==============================] - 28s - loss: 0.0639    
Epoch 955/1000
9990/9990 [==============================] - 28s - loss: 0.0593    
Epoch 956/1000
9990/9990 [==============================] - 28s - loss: 0.0413    
Epoch 957/1000
9990/9990 [==============================] - 28s - loss: 0.0528    
Epoch 958/1000
9990

9990/9990 [==============================] - 28s - loss: 0.1284    
Epoch 991/1000
9990/9990 [==============================] - 28s - loss: 0.0737    
Epoch 992/1000
9990/9990 [==============================] - 28s - loss: 0.0435    
Epoch 993/1000
9990/9990 [==============================] - 28s - loss: 0.0480    
Epoch 994/1000
9990/9990 [==============================] - 28s - loss: 0.0466    
Epoch 995/1000
9990/9990 [==============================] - 28s - loss: 0.0553    
Epoch 996/1000
9990/9990 [==============================] - 28s - loss: 0.0382    
Epoch 997/1000
9990/9990 [==============================] - 28s - loss: 0.0390    
Epoch 998/1000
9990/9990 [==============================] - 28s - loss: 0.0400    
Epoch 999/1000
9990/9990 [==============================] - 28s - loss: 0.0346    
Epoch 1000/1000
9990/9990 [==============================] - 28s - loss: 0.0360    


In [52]:
# load the network weights
model=load_model("Donna Files/text_generator_model.h5")

filename = "Donna Files/text_512_lstm_gen_weights-improvement-969-0.0264.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [126]:
# pick a random seed
start = np.random.randint(0, len(dataX)-1)
sentence="The weather is so nice today"
#pattern=sentence.split()
pattern = ["Hello",".","How", "are", "you",".","Where","are", "you", ".","You", "Suck"]
print "Seed:"
print "\"", ' '.join([str(value) for value in pattern]), "\""
indexed_input=[]
([indexed_input.append(bag_of_words[value.lower()]) for value in pattern])


Seed:
" Hello . How are you . Where are you . You Suck "


[None, None, None, None, None, None, None, None, None, None, None, None]

In [127]:
pattern=indexed_input

In [128]:
pattern+=list(np.zeros(int(math.ceil(len(pattern)/float(seq_length))*seq_length)-len(pattern)))

In [83]:
len(pattern)/float(seq_length)

1.2

In [87]:
math.ceil(len(pattern)/float(seq_length))

2.0

In [129]:
pattern

[4795,
 40,
 17,
 190,
 22,
 40,
 200,
 190,
 22,
 40,
 22,
 13087,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [53]:
n_vocab


110657

In [138]:
# generate characters
for i in range(10):
	x = np.reshape(pattern, (len(pattern)/seq_length, seq_length, 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = np.argmax(prediction)
	result = index_to_words[index]
	#seq_in = [index_to_words[value] for value in pattern]
	sys.stdout.write(result+" ")
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print "\nDone."

was uppity mean. go you tokyo.. ...right by i'm...god ...right 
Done.
